In [2]:
import pandas as pd

In [2]:
df = pd.read_excel('20210702_pb_data_postcode_1.xlsx')

In [3]:
df = pd.read_csv('ZP_RM_61k_with_address_base_estimated_misses.csv')

C:\Users\hp-de\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (37,43,53,71,77,144,145,146,147,148,149,150,151,152) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
needed_postcodes = list(df['POSTCODE_LOCATOR'].unique()) # this part is optional,if base is big, we can use this technique as
                                                         # an optimization method

In [7]:
dff

,Unnamed: 0,level_0,index,fullAddress,propertyId,__typename,postcode
0,0,0,0.0,"21 Hunters Way, Welwyn Garden City AL7 4NR",300935,Properties,AL7 4NR
1,1,1,1.0,"23 Hunters Way, Welwyn Garden City AL7 4NR",300936,Properties,AL7 4NR
2,2,2,2.0,"25 Hunters Way, Welwyn Garden City AL7 4NR",300937,Properties,AL7 4NR
3,3,3,3.0,"27 Hunters Way, Welwyn Garden City AL7 4NR",300938,Properties,AL7 4NR
4,4,4,4.0,"29 Hunters Way, Welwyn Garden City AL7 4NR",300939,Properties,AL7 4NR
...,...,...,...,...,...,...,...
1536202,1536202,50,NaN,"82 Hufling Lane, Burnley BB11 3QH",1451230,Properties,BB11 3QH
1536203,1536203,51,NaN,"83 Hufling Lane, Burnley BB11 3QH",30276375,Properties,BB11 3QH
1536204,1536204,52,NaN,"84 Hufling Lane, Burnley BB11 3QH",1451231,Properties,BB11 3QH
1536205,1536205,53,NaN,"86 Hufling Lane, Burnley BB11 3QH",1451232,Properties,BB11 3QH


In [6]:
dff = pd.read_csv('id_base_Nov29.csv')

In [8]:
def change_df(df,col_name):
    df['pk'] = '3.14'
    glob_a = {}
    def increment_postcode_if(x):
        if x in glob_a:
            glob_a[x] += 1
        else:
            glob_a[x] = 1
        return glob_a[x]
    
    df['pk'] = df[col_name].apply(lambda x:f'{x}_{increment_postcode_if(x)}') 
    df = df.set_index('pk')
    #df = df.drop('Unnamed: 0', axis=1)
    return df


def find_by_postcode(df, postcode):
    check = True
    cnt = 1
    finall_list = []
    while check:
        try:
            postcode_f = f'{postcode}_{cnt}'
            part = df.loc[postcode_f].to_dict()
    #         part = df1[df1['id'==f'{postcode}_{cnt}']].to_dict('records')
            finall_list += [part]
            cnt += 1
        except KeyError:
            check = False
    return pd.DataFrame(finall_list)


def exclude_postcode_from_address(address):
    try:
        return address.replace(re.findall(r'[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}', address)[0], '').strip()
    except IndexError:
        return 0

In [9]:
def to_str(x):
    try:
        a = str(int(x))
    except:
        a = x
    return a

def parse_base(allbase_kept):

    
    allbase_kept['BUILDING_NUMBER']=allbase_kept['BUILDING_NUMBER'].apply(lambda x:to_str(x))
    allbase_kept['SUB_BUILDING_NAME']=allbase_kept['SUB_BUILDING_NAME'].apply(lambda x:to_str(x))
    allbase_kept['BUILDING_NAME']=allbase_kept['BUILDING_NAME'].apply(lambda x:to_str(x))
    allbase_kept['BUILDING_NUMBER']=allbase_kept['BUILDING_NUMBER'].astype(str).apply(lambda x:x.lower())
    allbase_kept['BUILDING_NAME']=allbase_kept['BUILDING_NAME'].astype(str).apply(lambda x:x.lower())
    allbase_kept['SUB_BUILDING_NAME']=allbase_kept['SUB_BUILDING_NAME'].astype(str).apply(lambda x:x.lower())

    return allbase_kept

In [10]:
dff = change_df(dff,'postcode')

In [11]:
dff = dff.rename(columns={"fullAddress": "ADDRESS"})

In [12]:
df = parse_base(df) 

In [13]:
dff['ADDRESS'] = dff['ADDRESS'].apply(lambda x: ' '.join(x.lower().split(',')).strip())

In [14]:
en_ex = 0
rand_found = 0
def match(df,dff,index):
    global en_ex
    global rand_found
    p_code = df.iloc[index]['POSTCODE_LOCATOR']
    temp = find_by_postcode(dff,p_code)
    row = df.iloc[index]
    
    #print(temp['ADDRESS'])
#     print(row)
    
    for x in range(len(temp)):
        t = temp.iloc[x]['ADDRESS'].split(' ')
        t = [string for string in t if string != ""]
        #print(t)
        if t[0][0].isdigit() and row['BUILDING_NUMBER'] != 'nan':
            match = False
            if t[0].isdigit():
                t[0] = str(int(t[0]))
            else:
                t[0] = t[0].strip().lower()
            try:
                building_number = str(int(row['BUILDING_NUMBER'])).strip()
            except:
                building_number = ''
#             print(t[0])
            try:
                if  t[0] == building_number or t[0] == row['BUILDING_NAME'].strip().lower()\
                    or t[0] == row['SUB_BUILDING_NAME'].strip().lower():
                    #print(row['SUB_BUILDING_NAME'],',',temp.iloc[x]['ADDRESS'])
                    if (df.iloc[index]['SUB_BUILDING_NAME']) != 'nan':
                        if str(row['SUB_BUILDING_NAME']).strip().lower() in temp.iloc[x]['ADDRESS']:
                            #print(row['SUB_BUILDING_NAME'].strip().lower())
                            return temp.iloc[x]
                    else:
                        return temp.iloc[x]
                else:
    #                 print(str(row['BUILDING_NAME']),str(temp.iloc[x]['address']))
                    if str(row['BUILDING_NAME']).lower().replace(' ','') in str(temp.iloc[x]['ADDRESS']).lower().replace(' ','') and str(row['BUILDING_NAME']) != 'nan':
                        return temp.iloc[x]
            except:
                en_ex+=1
        else:
            for add in t:
                if add[0].isdigit():
                    digit = True
                    try:
                        building_number = str(int(row['BUILDING_NUMBER'])).strip()
                    except:
                        building_number = ''
                    try:
                        if  add == building_number or add == row['BUILDING_NAME'].strip().lower()\
                            or add == row['SUB_BUILDING_NAME'].strip().lower():
                            if (df.iloc[index]['SUB_BUILDING_NAME']) != 'nan':
                                if str(row['SUB_BUILDING_NAME']).strip().lower() in temp.iloc[x]['ADDRESS']:
                                    #print(row['SUB_BUILDING_NAME'].strip().lower())
                                    return temp.iloc[x]
                            else:
                                return temp.iloc[x]
                    except:
                        en_ex+=1
                        
    temp_1 = []            
    for x in range(len(temp)):
        t = temp.iloc[x]['ADDRESS'].split(' ')
        t = [string for string in t if string != ""]
        bn = str(row['BUILDING_NAME']).strip().lower()
        #print(t)
        if bn in t:
            temp_1.append(temp.iloc[x])
            #print("afasfasfasf")
        
        bn_spl = bn.split()
        #print(bn_spl)
        flag = True
        for i in bn_spl:
            if i in t:
                continue
            else:
                flag = False
                break
        
        if flag == True:
            temp_1.append(temp.iloc[x])
        
    
    
    temp_1 = pd.DataFrame(temp_1)
    #print('\n')
    #print(temp_1)
    if len(temp_1) == 0:
        return 'Not Found'
    if len(temp_1) == 1:
        return temp_1.iloc[0]
    
    for k in temp_1.index:
        #print("dfasdfasfasfasfasfasfasf")
        #print(temp_1['ADDRESS'])
        #print('\n')
        if row['SUB_BUILDING_NAME'] in temp_1['ADDRESS']:
            return temp1.iloc[k]
    for k in range(len(temp_1.index)):
        tt = temp_1.iloc[k]['ADDRESS'].split()
        bn = str(row['SUB_BUILDING_NAME']).strip().lower()
        #print(t)
        bn_spl = bn.split()
        #print(bn_spl)
        flag = True
        for i in bn_spl:
            if i in tt:
                continue
            else:
                flag = False
                break
        
        if flag == True:
            return temp_1.iloc[k]
    #print(temp_1.index, "adsfasfsaf")
    if len(temp_1.index)!=0:
        rand_found+=1
        return temp_1.iloc[0]

            
                
    return 'Not Found'

In [15]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Property type,Price,Source,Address,KeyFeatures,latitude,...,transport_code_2,transport_name_0,transport_name_1,transport_name_2,transport_type_0,transport_type_1,transport_type_2,transport_dist_0,transport_dist_1,transport_dist_2
0,0,0,0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,...,'210021502070','Pondfield Crescent','Marshalswick Lane','Marshalswick Lane','Bus','Bus','Bus',0.237281,0.299074,0.313605
1,1,1,1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,...,'43000320202','Harborne Junction','Harborne Junction','Harborne Junction','Bus','Bus','Bus',0.133796,0.137707,0.129067
2,2,2,2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,...,'43000429301','Bibsworth Avenue','Wake Green Rd','Thirlmere Drive','Bus','Bus','Bus',0.101797,0.148193,0.215372
3,3,3,3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,...,'43000432702','Westridge Rd','Westridge Rd','Dene Hollow','Bus','Bus','Bus',0.097220,0.105499,0.187090
4,4,4,4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,...,'210021690570','Daisy Drive','Mulberry Mead','Mulberry Mead','Bus','Bus','Bus',0.112517,0.192354,0.193202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,61118,61118,61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,...,'2905047','Seymour Avenue','New Town Road','Cambridge Street','Bus','Bus','Bus',0.064839,0.059859,0.070945
61119,61119,61119,61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,...,'13001573G','The Cricketers','The Cricketers','The Cricketers','Bus','Bus','Bus',0.094796,0.177964,0.179140
61120,61120,61120,61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,...,'3590E048000','Sutherland Drive','Sutherland Drive','Fieldhouse Drive','Bus','Bus','Bus',0.257807,0.243798,0.186875
61121,61121,61121,61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,...,'1900HA090300','Bridge Road','Bridge Road','Halimote Road','Bus','Bus','Bus',0.146733,0.175703,0.233379


In [16]:
dff

,Unnamed: 0,level_0,index,ADDRESS,propertyId,__typename,postcode
pk,,,,,,,
AL7 4NR_1,0,0,0.0,21 hunters way welwyn garden city al7 4nr,300935,Properties,AL7 4NR
AL7 4NR_2,1,1,1.0,23 hunters way welwyn garden city al7 4nr,300936,Properties,AL7 4NR
AL7 4NR_3,2,2,2.0,25 hunters way welwyn garden city al7 4nr,300937,Properties,AL7 4NR
AL7 4NR_4,3,3,3.0,27 hunters way welwyn garden city al7 4nr,300938,Properties,AL7 4NR
AL7 4NR_5,4,4,4.0,29 hunters way welwyn garden city al7 4nr,300939,Properties,AL7 4NR
...,...,...,...,...,...,...,...
BB11 3QH_51,1536202,50,NaN,82 hufling lane burnley bb11 3qh,1451230,Properties,BB11 3QH
BB11 3QH_52,1536203,51,NaN,83 hufling lane burnley bb11 3qh,30276375,Properties,BB11 3QH
BB11 3QH_53,1536204,52,NaN,84 hufling lane burnley bb11 3qh,1451231,Properties,BB11 3QH


In [17]:
match(df,dff,3)

Unnamed: 0                                            107
level_0                                               107
index                                                  22
ADDRESS       24 sandy croft  moseley  birmingham b13 0ep
propertyId                                         395272
__typename                                     Properties
postcode                                          B13 0EP
Name: 22, dtype: object

In [25]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Property type,Price,Source,Address,KeyFeatures,latitude,...,transport_name_0,transport_name_1,transport_name_2,transport_type_0,transport_type_1,transport_type_2,transport_dist_0,transport_dist_1,transport_dist_2,id
0,0,0,0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,...,'Pondfield Crescent','Marshalswick Lane','Marshalswick Lane','Bus','Bus','Bus',0.237281,0.299074,0.313605,267231
1,1,1,1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,...,'Harborne Junction','Harborne Junction','Harborne Junction','Bus','Bus','Bus',0.133796,0.137707,0.129067,452066
2,2,2,2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,...,'Bibsworth Avenue','Wake Green Rd','Thirlmere Drive','Bus','Bus','Bus',0.101797,0.148193,0.215372,394385
3,3,3,3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,...,'Westridge Rd','Westridge Rd','Dene Hollow','Bus','Bus','Bus',0.097220,0.105499,0.187090,395272
4,4,4,4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,...,'Daisy Drive','Mulberry Mead','Mulberry Mead','Bus','Bus','Bus',0.112517,0.192354,0.193202,27504935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,61118,61118,61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,...,'Seymour Avenue','New Town Road','Cambridge Street','Bus','Bus','Bus',0.064839,0.059859,0.070945,13789815
61119,61119,61119,61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,...,'The Cricketers','The Cricketers','The Cricketers','Bus','Bus','Bus',0.094796,0.177964,0.179140,6819504
61120,61120,61120,61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,...,'Sutherland Drive','Sutherland Drive','Fieldhouse Drive','Bus','Bus','Bus',0.257807,0.243798,0.186875,23797024
61121,61121,61121,61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,...,'Bridge Road','Bridge Road','Halimote Road','Bus','Bus','Bus',0.146733,0.175703,0.233379,9899488


In [21]:
df['id'] = '-'

In [22]:
res

NameError: name 'res' is not defined

In [23]:
ms = 0
fn = 0
for x in range(len(df.index)):
    if x%100 == 0:
        print(x,ms,fn)
        #df.to_csv('est_sold_match.csv')
    try:
        res = match(df,dff,x)
    except:
        ms+=1
        continue
    try:
        df.loc[x,'id'] = res['propertyId']
        fn+=1
        
    except:
        df.loc[x,'id'] = res
        

0 0 0
100 0 100
200 0 200
300 0 300
400 0 400
500 0 499
600 0 599
700 0 699
800 0 799
900 0 897
1000 0 997
1100 0 1097
1200 0 1197
1300 0 1296
1400 0 1396
1500 0 1495
1600 0 1595
1700 0 1693
1800 0 1793
1900 0 1893
2000 0 1992
2100 0 2092
2200 0 2192
2300 0 2292
2400 0 2392
2500 0 2492
2600 0 2592
2700 0 2692
2800 0 2791
2900 0 2891
3000 0 2990
3100 0 3089
3200 0 3187
3300 0 3287
3400 0 3387
3500 0 3486
3600 0 3584
3700 0 3684
3800 0 3783
3900 0 3883
4000 0 3982
4100 0 4082
4200 0 4179
4300 0 4279
4400 0 4379
4500 0 4476
4600 0 4575
4700 0 4674
4800 0 4773
4900 0 4873
5000 0 4972
5100 0 5072
5200 0 5171
5300 0 5271
5400 0 5371
5500 0 5471
5600 0 5571
5700 0 5670
5800 0 5769
5900 0 5869
6000 0 5969
6100 0 6069
6200 0 6169
6300 0 6268
6400 0 6367
6500 0 6467
6600 0 6565
6700 0 6665
6800 0 6765
6900 0 6863
7000 0 6963
7100 0 7062
7200 0 7162
7300 0 7261
7400 0 7361
7500 0 7461
7600 0 7560
7700 0 7660
7800 0 7758
7900 0 7857
8000 0 7956
8100 0 8056
8200 0 8156
8300 0 8255
8400 0 8355
8500 

60200 0 59931
60300 0 60030
60400 0 60130
60500 0 60230
60600 0 60330
60700 0 60428
60800 0 60526
60900 0 60625
61000 0 60725
61100 0 60825


In [27]:
dff

,Unnamed: 0,level_0,index,ADDRESS,propertyId,__typename,postcode
pk,,,,,,,
AL7 4NR_1,0,0,0.0,21 hunters way welwyn garden city al7 4nr,300935,Properties,AL7 4NR
AL7 4NR_2,1,1,1.0,23 hunters way welwyn garden city al7 4nr,300936,Properties,AL7 4NR
AL7 4NR_3,2,2,2.0,25 hunters way welwyn garden city al7 4nr,300937,Properties,AL7 4NR
AL7 4NR_4,3,3,3.0,27 hunters way welwyn garden city al7 4nr,300938,Properties,AL7 4NR
AL7 4NR_5,4,4,4.0,29 hunters way welwyn garden city al7 4nr,300939,Properties,AL7 4NR
...,...,...,...,...,...,...,...
BB11 3QH_51,1536202,50,NaN,82 hufling lane burnley bb11 3qh,1451230,Properties,BB11 3QH
BB11 3QH_52,1536203,51,NaN,83 hufling lane burnley bb11 3qh,30276375,Properties,BB11 3QH
BB11 3QH_53,1536204,52,NaN,84 hufling lane burnley bb11 3qh,1451231,Properties,BB11 3QH


In [24]:
df.to_csv('ZP_RM_61k_with_address_base_estimated_misses_with_id.csv')

In [48]:
df[df['id']!='Not Found']

,Unnamed: 0,Unnamed: 0.1,level_0,index,date,address,status,beds,type,agent,...,ALT_LANGUAGE,_merge,INSPECTION_DATE,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,TOTAL_FLOOR_AREA,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,NUMBER_HABITABLE_ROOMS,id
0,0,0,12,2191039,13.04.2021,"Berrichon Crescent, Whitehouse, Milton Keynes,...",Added,5,Detached House For Sale,Purplebricks,...,NaN,both,2017-03-30,86,92,196.0,74,45.0,NaN,32455762
1,1,1,40,1418831,04.06.2020,"Primrose Walk, Warminster",Added,3,Semi-Detached House For Sale,Cooper & Tanner,...,NaN,both,2020-05-21,69,86,68.0,222,97.0,5.0,1199004
2,2,2,118,2246204,08.03.2021,"Lamberhurst House, 13 Lovelinch Close, London,...",Reduced,3,Maisonette For Sale,Your Move,...,NaN,both,2010-12-21,75,81,74.57,207,153.0,3.0,21055134
3,3,3,155,985219,20.03.2020,"Main Street, Ingleby, Derby",Added,4,Equestrian Facility For Sale,Ashley Adams,...,NaN,both,2020-05-26,13,90,173.0,866,170.0,6.0,6627597
4,4,4,190,1693903,13.04.2021,"Queen Street, Coggeshall",Added,2,Terraced House For Sale,Philip James Estates,...,NaN,both,2019-06-08,71,87,63.0,187,63.0,3.0,5077659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,0,38,1358261,08.12.2020,"Wimblestone Road, WINSCOMBE",Added,3,Semi-Detached House For Sale,Farrons,...,NaN,both,2019-10-01,63,84,111.0,234,84.0,6.0,2867184
1996,1996,1,233,1135805,21.03.2020,"Church Street, Broadstairs",Added,2,Terraced House For Sale,Miles & Barr,...,NaN,both,2018-06-01,85,97,66.0,79,-15.0,NaN,32668837
1997,1997,2,436,2034719,11.06.2021,"The Hawnelands,Halesowen,B63 3RT",Added,3,Semi-Detached House For Sale,eXp UK,...,NaN,both,2012-05-03,58,75,127.0,250,144.0,6.0,803799
1998,1998,3,587,1648853,05.03.2021,"Swale Avenue, Peterborough",Added,3,Semi-Detached House For Sale,Newton Fallowell,...,NaN,both,2021-03-04,67,82,83.0,218,109.0,4.0,18003380


In [33]:
ind = 501

In [34]:
df.iloc[ind]['BUILDING_NAME']

'2 the old telephone exchange'

In [35]:
df.iloc[ind]['SUB_BUILDING_NAME']

'nan'

In [36]:
df.iloc[ind]['BUILDING_NUMBER']

'nan'

In [37]:
df.iloc[ind]['POSTCODE_LOCATOR']

'KT16 0DP'

In [38]:
df.iloc[ind]['address']

'The Courtyard, Pontefract, WF8'

In [26]:
df.iloc[500]['id']

19670529

In [20]:
df.to_csv('lilit_epc_school_transport_halfest.csv')

In [20]:
for x in dff['ADDRESS']:
    if 'gallions reach' in x:
        print(x)

In [25]:
list(find_by_postcode(dff,'DN10 4AN')['ADDRESS'])

['beacon view  fountain hill road  walkeringham  doncaster dn10 4lt',
 'carrfield farm  fountain hill road  walkeringham  doncaster dn10 4lt',
 'cedar house  fountain hill road  walkeringham  doncaster dn10 4lt',
 'eastlyn  fountain hill road  walkeringham  doncaster dn10 4lt']